GET A MPS WITH NATIVE BOND DIMENSION 2

In [1]:
import numpy as np
from ncon import ncon
from numpy import linalg as LA 
from scipy.linalg import null_space as NS 

In [2]:
N = 3
mps_GHZ = np.zeros((2**N))
mps_GHZ[0] = 1/np.sqrt(2)
mps_GHZ[-1] = 1/np.sqrt(2)

print(mps_GHZ.reshape(2,2,2))

[[[0.70710678 0.        ]
  [0.         0.        ]]

 [[0.         0.        ]
  [0.         0.70710678]]]


In [13]:
mps_sites = [0 for _ in range(N)]


Atemp, Stemp, Vhtemp = LA.svd(mps_GHZ.reshape(2,2**(N-1)), full_matrices=False)
mps_sites[0] = Atemp.reshape(1,2,2).transpose(0,2,1)  #shape della matrice A_1 ---> A_8 paper è 2,2 - mapping 1 a 1
mps_sites[1] = (np.diag(Stemp)@Vhtemp)/LA.norm(Stemp)

for i in range(1,N-1):
    Atemp, Stemp, Vhtemp = LA.svd(mps_sites[i].reshape(2**2,2**(N-i-1)), full_matrices=False)
    mps_sites[i] = Atemp[:,:2].reshape(2,2,2).transpose(0,2,1)
    mps_sites[i+1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,-1))/LA.norm(Stemp)

mps_sites[-1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,2,1)).transpose(0,2,1)/LA.norm(Stemp)

mps_2 = ncon([mps_sites[0], mps_sites[1], mps_sites[2]],[[-1,1,-2],[1,2,-3],[2,-4,-5]])
print("Error from MPS to MPS(2):\n",LA.norm(mps_GHZ.reshape(2,2,2) - mps_2[0,:,:,0,:]) / LA.norm(mps_GHZ))

#ora l'ordinamento degli indici è A[j,k,l] con j,k indici virtuali e l indice fisico
#nella contrazione quindi si seguirà lo schema A_jkl A_jkl' = A_ll' con l,l' indici fisici corrispondenti al 1° e 2° sito


Error from MPS to MPS(2):
 1.5700924586837754e-16


In [35]:


print(mps_sites[0].shape)
k_a1 = NS(mps_sites[0].transpose(0,2,1).reshape(2,2).conj().T)
print(mps_sites[1].shape)
k_a2 = NS(mps_sites[1].transpose(0,2,1).reshape(4,2).conj().T)
print(mps_sites[2].shape)
k_a3 = NS(mps_sites[2].transpose(0,2,1).reshape(4,1).conj().T)
print(k_a3)

(1, 2, 2)
(2, 2, 2)
(2, 1, 2)
[[0.         0.         0.70710678]
 [1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         0.70710678]]


In [31]:
for k in null_spaces:
    print("----")
    print("Null space:\n",k)
    print("Shape:\n",k.shape)
    print("Identity:\n",k.conj().T@k)

----
Null space:
 []
Shape:
 (2, 0)
Identity:
 []
----
Null space:
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]]
Shape:
 (4, 2)
Identity:
 [[1. 0.]
 [0. 1.]]
----
Null space:
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]]
Shape:
 (4, 2)
Identity:
 [[1. 0.]
 [0. 1.]]
